In [1]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [3]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

In [4]:
from sklearn.metrics import confusion_matrix
import itertools
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')
    print metrics.precision_recall_fscore_support(test_y_list, predict_y_list)
    print confusion_matrix(test_y_list, predict_y_list)

In [5]:
# z-score normalization
# scaler = preprocessing.StandardScaler().fit(train_x_list)
# train_x_list = scaler.transform(train_x_list)
# test_x_list = scaler.transform(test_x_list)

In [6]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [7]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [8]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [9]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

# ENN down-sampling training data
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN
enn = EditedNearestNeighbours()
# enn = SMOTEENN()

train_x_list_enn, train_y_list_enn = enn.fit_sample(train_x_list, train_y_list)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
train_y_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list_enn)):
    train_x_lists[int(train_y_list_enn[i])].append(train_x_list_enn[i])
    train_y_lists[int(train_y_list_enn[i])].append(train_y_list_enn[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

train_x_lists_enn = train_x_lists
train_y_lists_enn = train_y_lists

/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
No handlers could be found for logger "imblearn.base"


number of class 0 : 3
number of class 1 : 59
number of class 2 : 1373
number of class 3 : 674
number of class 4 : 2


In [10]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
train_y_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list)):
    train_x_lists[int(train_y_list[i])].append(train_x_list[i])
    train_y_lists[int(train_y_list[i])].append(train_y_list[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i]), len(train_y_lists[i])
print '------------------'

train_x_lists[2] = train_x_lists_enn[2]
train_y_lists[2] = train_y_lists_enn[2]

train_x_list = np.vstack(train_x_lists)
train_y_list = np.concatenate(train_y_lists)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list)):
    train_x_lists[int(train_y_list[i])].append(train_x_list[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

number of class 0 : 881 881
number of class 1 : 2093 2093
number of class 2 : 5590 5590
number of class 3 : 674 674
number of class 4 : 721 721
------------------
number of class 0 : 881
number of class 1 : 2093
number of class 2 : 1373
number of class 3 : 674
number of class 4 : 721


In [11]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

(5742,) (5742, 384)
(8257,) (8257, 384)


In [12]:
print train_x_list[0][:10]
print test_x_list[0][:10]

[  1.01766700e-01   3.14376600e-04   1.01452300e-01   5.80000000e+01
   2.50000000e+01   1.67105400e-02  -6.80357300e-05   2.35821500e-02
   6.06719100e-04   2.49522500e-02]
[  1.23016500e-02   9.11361400e-05   1.22105100e-02   5.20000000e+01
   2.40000000e+01   1.31725500e-03   3.23349300e-07   1.29866300e-03
   6.89166500e-06   2.62522000e-03]


In [13]:
X = train_x_list
y = train_y_list

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis

names = [
    "Nearest Neighbors",
    "RBF SVM",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
    "LDA"
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(), 
    LinearDiscriminantAnalysis()
]

In [15]:
for name, clf in zip(names, classifiers):
    print 'Classifier:', name

    clf.fit(train_x_list, train_y_list)
    predict_y_list = clf.predict(test_x_list)
    calc_score(test_y_list, predict_y_list)

    print '-------------------------------'

Classifier: Nearest Neighbors
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.21357951717603677, 0.22300935253020787, 0.15470206230849584, None)
weighted:  (0.50192197560472329, 0.24512534818941503, 0.24904306447567109, None)
(array([ 0.09090909,  0.19895407,  0.69633508,  0.        ,  0.08169935]), array([ 0.31587561,  0.58023873,  0.17314488,  0.        ,  0.04578755]), array([ 0.14118508,  0.29630884,  0.27733095,  0.        ,  0.05868545]), array([ 611, 1508, 5377,  215,  546]))
[[ 193  322   66    7   23]
 [ 362  875  198   13   60]
 [1380 2817  931   64  185]
 [  42  110   50    0   13]
 [ 146  274   92    9   25]]
-------------------------------
Classifier: RBF SVM
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.036526583504904932, 0.20000000000000001, 0.061771633384536603, None)
weighted:  (0.033354782563519828, 0.18263291752452465, 0.056407668126366238, None)
(array([ 0.        ,  0.18263292,  0.        ,  0.        ,  0.        ]), ar

/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28891604452474973, 0.33669608757404501, 0.27343151533999621, None)
weighted:  (0.57062060795593639, 0.38319002058859147, 0.41153277013112094, None)
(array([ 0.20541205,  0.27209172,  0.76146045,  0.09318996,  0.11242604]), array([ 0.27332242,  0.64522546,  0.34907941,  0.24186047,  0.17399267]), array([ 0.23455056,  0.38276947,  0.47870441,  0.13454075,  0.13659238]), array([ 611, 1508, 5377,  215,  546]))
[[ 167  296   52   36   60]
 [ 163  973  270   40   62]
 [ 430 2112 1877  357  601]
 [   5   30  101   52   27]
 [  48  165  165   73   95]]
-------------------------------
Classifier: Random Forest
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28040438871863421, 0.24567152027383438, 0.15545857417298409, None)
weighted:  (0.56318284703073451, 0.28303257841831175, 0.24155443964908183, None)
(array([ 0.31147541,  0.20839342,  0.76636364,  0.11578947,  0.        ]), array([ 0.06219313,  0.9582228

/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.39784390412029091, 0.2915279816885451, 0.25480297974296934, None)
weighted:  (0.56655737443549226, 0.52077025554075329, 0.52483612287835779, None)
(array([ 0.15528313,  0.31860465,  0.72366507,  0.66666667,  0.125     ]), array([ 0.43535188,  0.36339523,  0.64775897,  0.00930233,  0.0018315 ]), array([ 0.22891566,  0.33952912,  0.68361138,  0.01834862,  0.00361011]), array([ 611, 1508, 5377,  215,  546]))
[[ 266  151  194    0    0]
 [ 319  548  640    0    1]
 [1009  878 3483    1    6]
 [  28   42  143    2    0]
 [  91  101  353    0    1]]
-------------------------------
Classifier: LDA
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.31739575084824923, 0.39036745872701817, 0.30833613442513275, None)
weighted:  (0.60142750488151797, 0.4051108150660046, 0.43616887456552639, None)
(array([ 0.26628571,  0.30625   ,  0.79200653,  0.11664075,  0.10579577]), array([ 0.38134206,  0.64986737,  0.361167

/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [16]:
# z-score normalization
scaler = preprocessing.StandardScaler().fit(train_x_list)
train_x_list = scaler.transform(train_x_list)
test_x_list = scaler.transform(test_x_list)

for name, clf in zip(names, classifiers):
    print 'Classifier:', name

    clf.fit(train_x_list, train_y_list)
    predict_y_list = clf.predict(test_x_list)
    calc_score(test_y_list, predict_y_list)

    print '-------------------------------'

Classifier: Nearest Neighbors
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.26923006977491665, 0.29713868281828548, 0.22231302314409035, None)
weighted:  (0.54514496644606669, 0.30785999757781274, 0.32613314857922077, None)
(array([ 0.12137097,  0.24791006,  0.73828125,  0.14124294,  0.09734513]), array([ 0.49263502,  0.57029178,  0.24604798,  0.11627907,  0.06043956]), array([ 0.19475898,  0.34558971,  0.36908913,  0.12755102,  0.07457627]), array([ 611, 1508, 5377,  215,  546]))
[[ 301  224   46    7   33]
 [ 374  860  238    6   30]
 [1573 2140 1323  116  225]
 [  60   57   55   25   18]
 [ 172  188  130   23   33]]
-------------------------------
Classifier: RBF SVM
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.3605814533163173, 0.39624981265346193, 0.33534003027017384, None)
weighted:  (0.61079607172323935, 0.46178999636671914, 0.47965038426328793, None)
(array([ 0.30537353,  0.28845668,  0.79058663,  0.17435897,  0.24413146]), array([

In [17]:
print predict_y_list[:100]

[4 2 4 4 2 0 4 1 1 4 3 2 3 4 1 1 1 2 2 4 2 2 0 1 4 4 0 2 4 1 3 2 3 2 4 1 1
 4 2 4 1 4 4 2 2 1 1 1 1 1 2 4 0 4 1 4 2 2 0 1 2 2 4 3 1 1 1 1 2 2 1 2 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 4 2 4 2 2 4 1 1 1]


In [18]:
print test_y_list[:100]

[2 2 2 2 2 4 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1
 1 2 2 2 2 2 2 2 1 1 2 1 1 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2
 1 1 1 2 1 1 1 1 1 2 2 1 1 1 2 1 1 2 2 2 2 2 1 1 1 2]
